## Notebook to compute total travel demand (i.e., cumulative for all modes) by TAZ for a given scenario.

This is a "large sledgehammer" and breaks down and reports demand only in these four cumulative categories:
auto, non-motorized, transit, and truck.

In [ ]:
# Notebook to compute total travel demand (i.e., cumulative for all modes) by TAZ for a given scenario.
# This is a "large sledgehammer" and breaks down and reports demand only in these four cumulative categories:
# auto, non-motorized, transit, and truck.

import openmatrix as omx
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import bokeh
import xarray as xr
import hvplot.pandas
import hvplot.xarray
import json

In [ ]:
%matplotlib notebook

### User input required: Specify the locations of input and output direcotories:

In [ ]:
%run "s:/jupyter_notebooks/config.py"

### User input required: path to root directory of scenario to use for the current run of this notebook:

In [ ]:
# ===>>> USER INPUT REQUIRED: <<<===
#
# Supply path to root directory of scenario to use for the current run of this notebook:
# 
home_dir = base_scenario_dir

### User input required: Name of CSV report file generated by this notebook

In [ ]:
# ===>>> USER INPUT REQUIRED: <<<===
#
# Supply name of CSV output file for tabular results generated by this notebook:
#
csv_output_fn = 'taz_all_modes_report_base.csv'

In [ ]:
taz_shapefile_base_dir = reference_data_dir +  'canonical_TAZ_shapefile/'

In [ ]:
# trip_tables directory - this really "should" be a subdirectory of the base directory, but is isn't currently.
# The real McCoy - where things should go, and will eventually go
tt_dir = home_dir + 'out/'

In [ ]:
# trip tables OMX file (matrices)
tt_am = tt_dir + 'AfterSC_Final_AM_Tables.omx'
tt_md = tt_dir + 'AfterSC_Final_MD_Tables.omx'
tt_pm = tt_dir + 'AfterSC_Final_PM_Tables.omx'
tt_nt = tt_dir + 'AfterSC_Final_NT_Tables.omx'
trip_tables = { 'am' :  omx.open_file(tt_am, 'r'),
                'md' : omx.open_file(tt_pm, 'r'),
                'pm' : omx.open_file(tt_pm,'r'),
                'nt'  : omx.open_file(tt_nt, 'r') }

In [ ]:
num_tazes = trip_tables['am'].shape()[0]

In [ ]:
# Mapping from TAZ-ID to OMX index for the 4 periods (these *should* be the same)
taz_to_omxid_am = trip_tables['am'].mapping('ID')
taz_to_omxid_am = trip_tables['md'].mapping('ID')
taz_to_omxid_pm = trip_tables['pm'].mapping('ID')
taz_to_omxid_nt =  trip_tables['nt'].mapping('ID')

In [ ]:
# We'll assume that the mapping from TAZ ID to OMX ID doesn't vary by time period.
# We'll use the AM mapping as _the_ mapping for all time periods, pending confirmation.
# 
# TBD: Insert "sanity check" that the 4 mappings on "ID" are identical.
#
taz_to_omxid = taz_to_omxid_am

In [ ]:
# Function: tt_total_for_mode
#
# Summary: Return the total travel demand, over all i and j, from TAZ[i] to TAZ[j] for the specified mode.
#
# Given the OMX trip tables for the 4 time periods and a mode,
# return an numpy array with the "numpy sum" of the data in the OMX array for the 4 time periods.
def tt_total_for_mode(tts, mode):
	am = tts['am'][mode]
	md = tts['md'][mode]
	pm = tts['pm'][mode]
	nt =  tts['nt'][mode]
	# Convert OMX arrays into numpy arrays
	am_np = np.array(am)
	md_np = np.array(md)
	pm_np = np.array(pm)
	nt_np = np.array(nt)
	total = am_np + md_np + pm_np + nt_np
	return total
# end_def tt_total_for_mode

In [ ]:
# Function to generate the calculation to total demand for a list of modes.
# Return a dictionary containing the total demand for each mode, with the key value == the mode name
def tt_totals_for_mode_list(tts, mode_list):
    retval = {}
    for mode in mode_list:
        temp = tt_total_for_mode(tts, mode)
        retval[mode] = temp
    #
    return retval
# end_def tt_total_for_mode_list

### Load trip tables for auto mode; calculate per-TAZ demand for this mode

In [ ]:
# Auto mode
all_auto = tt_totals_for_mode_list(trip_tables, ['SOV', 'HOV'])
sov = all_auto['SOV']
hov = all_auto['HOV']
sov_total = sov.sum(axis=1)
hov_total = hov.sum(axis=1)
# Grand total for the 'auto' mode
auto_total = sov_total + hov_total

### Load trip tables for non-motorized mode; calculate per-TAZ demand for this mode

In [ ]:
# Non-motorized mode
all_nm = tt_totals_for_mode_list(trip_tables, ['Walk', 'Bike'])
walk = all_nm['Walk']
bike = all_nm['Bike']
walk_total = walk.sum(axis=1)
bike_total = bike.sum(axis=1)
# Grand total for the 'non-motorized' mode
nm_total = walk_total + bike_total

### Load trip tables for truck mode; calculate per-TAZ demand for this mode

In [ ]:
# Truck mode
all_truck = tt_totals_for_mode_list(trip_tables, ['Heavy_Truck', 'Heavy_Truck_HazMat', 
                                                  'Medium_Truck', 'Medium_Truck_HazMat', 'Light_Truck'])
heavy = all_truck['Heavy_Truck']
heavy_haz = all_truck['Heavy_Truck_HazMat']
medium = all_truck['Medium_Truck']
medium_haz = all_truck['Medium_Truck_HazMat']
light = all_truck['Light_Truck']
heavy_total = heavy.sum(axis=1)
heavy_haz_total = heavy_haz_total = heavy_haz.sum(axis=1)
medium_total = medium.sum(axis=1)
medium_haz_total = medium_haz.sum(axis=1)
light_total = light.sum(axis=1)
# Grand total for the 'truck' mode
truck_total = heavy_total + heavy_haz_total + medium_total + medium_haz_total + light_total

### Load trip tables for transit mode; calculate per-TAZ demand for this mode

In [ ]:
# Transit mode
all_transit = tt_totals_for_mode_list(trip_tables,[ 'DAT_Boat', 'DET_Boat', 'DAT_CR', 'DET_CR', 
                                                    'DAT_LB', 'DET_LB', 'DAT_RT', 'DET_RT', 'WAT'])
dat_boat = all_transit['DAT_Boat']
det_boat = all_transit['DET_Boat']
dat_cr = all_transit['DAT_CR']
det_cr = all_transit['DET_CR']
dat_lb = all_transit['DAT_LB']
det_lb = all_transit['DET_LB']
dat_rt = all_transit['DAT_RT']
det_rt = all_transit['DET_RT']
wat = all_transit['WAT']
dat_boat_total = dat_boat.sum(axis=1)
det_boat_total = det_boat.sum(axis=1)
dat_cr_total = dat_cr.sum(axis=1)
det_cr_total = det_cr.sum(axis=1)
dat_lb_total = dat_lb.sum(axis=1)
det_lb_total = det_lb.sum(axis=1)
dat_rt_total = dat_rt.sum(axis=1)
det_rt_total = det_rt.sum(axis=1)
wat_total = wat.sum(axis=1)
# Grand total for transit mode
transit_total = dat_boat_total + det_boat_total + dat_cr_total + det_cr_total + \
              dat_lb_total + det_lb_total + dat_rt_total + det_rt_total + wat_total

### Calculate grand total per-TAZ demand

In [ ]:
# Grand grand total (i.e., across all modes) of demand by TAZ of origin
grand_total = auto_total + nm_total + truck_total  + transit_total

In [ ]:
# Build a data frame, indexed by omxid, containing the total of the following kinds of trips originating in each TAZ:
# auto_trips, truck_trips, non-motorized trips, transit trips, and total trips.
total_trips_df = pd.DataFrame(grand_total, columns=['total_trips'])
total_trips_df['total_auto'] = auto_total
total_trips_df['total_truck'] = truck_total
total_trips_df['total_nm'] = nm_total
total_trips_df['total_transit'] = transit_total 
# Set the data frame's index to the omxid of each row, i.e., its index
total_trips_df['omxid'] = total_trips_df.index
total_trips_df.set_index('omxid')

In [ ]:
# Load the candidate canonical TAZ shapefile as a geopands dataframe.
taz_shapefile = taz_shapefile_base_dir + 'candidate_CTPS_TAZ_STATEWIDE_2019_wgs84.shp'
taz_gdf = gp.read_file(taz_shapefile)
taz_gdf.set_index("id")

In [ ]:
# Add a 'omxid' column to the TAZ geodataframe, in prep for joining with the total trips dataframes.
# ==> This also can be done earlier.
taz_gdf['omxid'] = taz_gdf.apply(lambda row: taz_to_omxid[row.id], axis=1)

In [ ]:
# Join the shapefile geodataframe to the total trips dataframe on 'omxid'
joined_df = taz_gdf.join(total_trips_df.set_index('omxid'), on='omxid')

In [ ]:
joined_df

### Generate CSV report output

In [ ]:
# Export the useful columns of data in the 'joined_df' dataframe as a CSV file
fq_output_fn = sandbox_dir + csv_output_fn
joined_df.to_csv(fq_output_fn, sep=',', 
                  columns=['id', 'town', 'state', 'total_trips', 'total_auto', 'total_nm', 'total_truck', 'total_transit'])

In [ ]:
# Free as much memory as possible in order to allow the generation of maps and plots to execute
# as quickly as possible, and in some cases, to execute at all.
#
del all_auto
del all_truck
del all_nm 
del all_transit 

### Make static and interactive maps of the results

In [ ]:
# Make a static map of total trips by origin TAZ
joined_df.plot("total_trips", figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.title('Total Trips by Origin TAZ')
plt.show()

In [ ]:
# Make an interactive map of total trips by origin TAZ
joined_df.hvplot(c='total_trips', 
                 geo=True, 
                 hover_cols=['id', 'town_state', 'total_trips', 'total_auto', 'total_nm', 'total_truck', 'total_transit'], 
                 clabel='Total Trips', 
                 cmap='plasma',
                 frame_height=500).opts(title='Total Trips by Origin TAZ')

### Make a bar chart of number of trips by mode (mode-share)

In [ ]:
# Make a static bar chart of number of trips by mode (mode-share)
# N.B. ttdf == shorthand for total_trips_df
ttdf = total_trips_df
tot_trips = [ttdf['total_auto'].sum() / 10e6, 
             ttdf['total_truck'].sum() / 10e6, 
             ttdf['total_nm'].sum() / 10e6, 
             ttdf['total_transit'].sum() / 10e6]
modes = ['Auto', 'Truck', 'Non-motorized', 'Transit']
temp = { 'modes' : modes, 'total_trips' : tot_trips }
temp_df = pd.DataFrame(temp)
temp_df.plot.bar(x='modes', y='total_trips', ylabel='Number of Trips x 10**6', title='Mode Share')